In [ ]:
!pip install pypdf
!pip install transformers einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama_index
!pip install llama-index-llms-huggingface
!pip install langchain-community langchain-core
!pip install llama-index-embeddings-langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 17.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (73

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
from huggingface_hub import login
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:

# Load documents
documents = SimpleDirectoryReader("/content/").load_data()

#prompt
system_prompt = """
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
query_wrapper_prompt = SimpleInputPrompt("{query_str}")


In [ ]:
login(token="hf_token") #HuggingFace login

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# create llm
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True}
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

index = VectorStoreIndex.from_documents(documents, service_context=service_context)

#query engine
query_engine = index.as_query_engine()


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-6-15df54d75515>:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [ ]:
response = query_engine.query("how to deal with nosebleed?")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



To treat a common nosebleed, follow these steps:

1. Sit up and lean forward. Keep the head up and lean forward so the blood doesn't go down the throat. This could cause you to choke or have an upset stomach.
2. Gently blow your nose. This will clear any blood clots.
3. Pinch the nose. Use the thumb and a finger to pinch both nostrils shut. Breathe through the mouth. Keep pinching for 10 to 15 minutes. Pinching puts pressure on the blood vessels and helps stop the blood flow.
4. If the bleeding doesn't stop, pinch the nose again for up to 15 minutes. Don't let go for at least five minutes even to check if the bleeding has stopped. Seek emergency care if the bleeding doesn't stop after the second try.
5. Prevent another nosebleed. Don't pick or blow the nose. And don't drop the head below the heart or lift anything heavy for many hours. Gently put a saline gel, antibiotic ointment, or petroleum jel


In [ ]:
response = query_engine.query("what to do in case of a fracture?")
print(response)


In case of a fracture, it is important to call 911 if the person is experiencing any of the following symptoms: bleeding a lot, is unresponsive, is not breathing, or has more than one injury. Additionally, if the fracture is a compound fracture (a broken bone that has pierced the skin), or if the person is experiencing coldness or discoloration in the affected area, it is important to seek immediate medical attention.

For a closed fracture (a fracture where the bone is not pierced through the skin), you can use ice packs to assist with pain relief and swelling. However, do not try to straighten the bone. Instead, use a splint and padding to keep the limb still, and elevate it to reduce swelling. Give the person anti-inflammatory drugs such as Advil (ibuprofen) or Aleve (naproxen) for pain relief.

It is important to remember that fractures can be serious injuries that require prompt medical attention. If you are unsure of the severity of the fracture or if the person


In [ ]:
response = query_engine.query("The patient is bleeding excessively.")
print(response)



In this situation, it is crucial to act quickly and efficiently to stop the bleeding and prevent further complications. Here are the steps you should take:

1. Check for the source of the bleeding: Before doing anything else, locate the source of the bleeding and assess its severity. If the bleeding is severe, call 911 or your local emergency number immediately.
2. Apply direct pressure: If the bleeding is not severe, apply direct pressure to the wound using a clean cloth or gauze. Press firmly on the wound to stop the bleeding.
3. Elevate the affected area: If possible, elevate the affected area above the person's heart to reduce blood flow to the area and slow down the bleeding.
4. Add more bandages: If the bleeding does not stop with direct pressure, add more bandages on top of the existing one. Press firmly on the area to stop the bleeding.
5. Use a tourniquet: If the bleeding is severe and does not stop with direct pressure and bandaging, consider using a tourniquet. A tournique

In [ ]:
response = query_engine.query("steps to take for a heatstroke")
print(response)


To treat someone with heatstroke, follow these steps:

1. Move the person out of the heat source and into a cooler location.
2. Call 911 or your local emergency number for immediate medical help.
3. Remove excess clothing and sponge the person with cool water.
4. Use a fan or misting device to cool the person down.
5. Place ice packs or cool, wet towels on the neck, armpits, and groin.
6. Monitor the person's vital signs and watch for any changes in their condition.
7. Keep the person as cool and comfortable as possible until medical help arrives.

Remember, heatstroke can be life-threatening, so it's important to act quickly and seek medical attention right away if you suspect someone has heatstroke.
